# Quickstart for laminar, non-Newtonian pipe flow

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rheopy/rheoflow/blob/master/notebooks/index.ipynb)

In [1]:
run_from_colab = True
rheoflow_installation = 'rheoflow not installed' 
if (rheoflow_installation == 'rheoflow not installed' or run_from_colab == True):
    !pip3 install git+https://github.com/rheopy/rheoflow.git
    !pip install ipywidgets

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, Math, Latex
import rheoflow

In [2]:
def tube_dashboard_plot(pressure_drop,radius,length,density,eta0,reltime,n):
    plt.figure(figsize=(14,12))
    viscosity = rheoflow.viscosity.carreau('Carreau dashboard',eta0=eta0,etainf=.1,reltime=reltime,a=2.0,n=n)
    pipe = rheoflow.pipe.laminar('my laminar tube',density=density,radius=radius,length=length,
                              viscosity=viscosity)
    pipe.pressure_drop = pressure_drop
    plt.subplot(221)
    viscosity.visc_plot()
    plt.plot(pipe.shear_rate_wall,pipe.viscosity_wall(),'o')
    plt.subplot(222)
    pipe.q_plot(pressure_drop_min=200.,pressure_drop_max=200000.)
    plt.plot(pipe.q,pipe.pressure_drop,'o')
    plt.subplot(223)
    pipe.shear_rate_plot()
    plt.subplot(224)
    pipe.vz_plot()
    
#tube_dashboard_plot(pressure_drop=20000.,radius=.01,length=1.,density=1000.,eta0=10.,reltime=1.,n=.6)
interact(tube_dashboard_plot,pressure_drop=(100.,10000.),radius=(.01,.05),length=(.1,.5),
         density=(500,1500),eta0=(1,100),reltime=(.001,10),n=(.2,0.9),continuous_update=False)

interactive(children=(FloatSlider(value=5050.0, description='pressure_drop', max=10000.0, min=100.0), FloatSli…

<function __main__.tube_dashboard_plot(pressure_drop, radius, length, density, eta0, reltime, n)>

## Nomenclature

This notebook provides deep examples for using rheopy for pipe flow.  Both tube and slit flow examples with a variety of viscosity models are provided.  SI units units are used everywhere.  For the variables used here, the units are:

$\eta = \left< Pas \right>$

$\sigma = \left< Pa \right>$

$\dot{\gamma} = \left< s^{-1} \right>$

$\Delta P = \left< Pa \right>$

$Q = \left< \frac{m^3}{s} \right>$

$v_z = \left< \frac{m}{s} \right>$

$length = \left< m \right>$

$radius = \left< m \right>$

## Calculation procedure

The rheopy.laminar notebook is intended to work with any viscosity model.  There are four elemenets of computation:

1. A viscosity model using the class structure provided - $\eta(\dot{\gamma})$
2. Shear rate calculation at any radial position (tube) or height (slit) - $\dot{\gamma}(r)$
3. Velocity calculation at any position - $v_z(r)$
4. Volumetric flow rate - $Q$, given $\Delta P$ (or vice versa)

## Details of calculations

### 1 - Viscosity model
Use pre-existing, or create new function with viscosity as a function of shear rate:

$\eta \left[ \dot{\gamma} \right]$

### 2 - Shear rate calculation ($\dot{\gamma}$)
The equation to be solved for shear rate at any radial position is

$\eta \left[ \dot{\gamma}(r) \right] \dot{\gamma}(r) = \frac{r}{2} \frac{\Delta P}{L}$,

where $\dot{\gamma}(r) = \frac{d v_z(r)}{dr}$

This equation is simply an equality of shear stress from constituitve model and shear stress from a momentum balance.  This calculation is numerical in rheopy because this equation does not always have analytical solutions.  The Carreau viscosity model is an example where an analytical solution does not yet exist.  

Note that 4 inputs are required: 
1. Viscosity model - $\eta(\dot{\gamma})$
2. Tube radius
3. Tube length
4. Pressure drop ($\Delta P$)

### 3 - Velocity calculation ($v_z$)
The axial velocity is calculated from the shear rate directly

$v_z(r) = -\int_{r}^{R} \dot{\gamma}(r)dr$

This integral is computed numerically beacuse the shear rate is computed numerically.

### 4 - Volumetric flow rate ($Q$)
The volumetric flow rate is computed by numerically integrating the velocity profile

$Q = 2\pi \int_{0}^{R} v_z(r)rdr$

Note that if Q is an input, then $\Delta P$ must be solved for iteratively.  